In [1]:
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False
IN_COLAB

debug=False
DOWNLOAD_DATA = False

### Data Preparation
- Download of Dataset 
**P.S**: Randomly Sampled 10 instances from each target class as described in the paper.
- Option 1: Download from Archive.org
    - [Archive Link](https://archive.org/details/Imagenet_NAG)
    - [train.zip](https://archive.org/download/Imagenet_NAG/train.zip)
    - [valid.zip](https://archive.org/download/Imagenet_NAG/valid.zip)
- Option 2 : Mega Download Link for Train abd Validation data of Imagenet 2012 (Obtained from Kaggle)
    - Validation Data: [Mega Link](https://mega.nz/#!yDoTDIyD!RjN6OBA92-KLpNqDeLS3OzwmAYesEbTsiQat9hT6p6s)
    - Trainning Data: [Mega Link](https://mega.nz/#!vKY0WSDa!4aibnBkiXUrO9MkhQlLGXac7wLF5HY7O4LzfdFEaeQU) 
<!-- - If link fails to work use the following Colab notebook to generate your own subset of trainning examples. [Link](https://colab.research.google.com/drive/1LbZBfgqntWb3HuC3UFyF_FvwnHtd1xTA) -->
- Setting up of Folder Structure
For Easier handling and reproducibility of results download from mega link 


In [2]:
if IN_COLAB or DOWNLOAD_DATA:
    !mkdir ILSVRC
    !sudo apt install aria2 zip -y
    !aria2c -x 4 https://archive.org/download/Imagenet_NAG/train.zip -o train.zip
    !aria2c -x 4 https://archive.org/download/Imagenet_NAG/valid.zip -o valid.zip
    !wget https://archive.org/download/Imagenet_NAG/LOC_val_solution.csv -O ILSVRC/LOC_val_solution.csv
    !wget https://archive.org/download/Imagenet_NAG/LOC_synset_mapping.txt -O ILSVRC/LOC_synset_mapping.txt
    !unzip -qq train.zip -d ILSVRC/ 
    !unzip -qq valid.zip -d ILSVRC/
    
    
    # TODO Download Data from Archive
    # Extract and Do the Pre-Processing
#     !rm train.zip
#     !rm valid.zip
    

In [3]:
from glob import glob

train_ok = True
val_ok = True
print("Training Data Verification")
cls_count = len(glob("ILSVRC/train/*"))
print("Total Number of Classes: {} in train directory".format(cls_count))
count = 0
for cls_ in glob("ILSVRC/train/*"):
    imgs = glob(cls_ + "/*")
    img_count = len(imgs)
    count += img_count
    if img_count != 10:
        print(cls_.split("/")[-1], img_count)
        train_ok=False
print("Total {} number of files in {} classes. i.e 10 Images/Class".format(count, cls_count))

print("Validation Data Verification")
val_files = glob("ILSVRC/valid/*")
val_count = len(val_files)
if val_count == 50000:
    print("Validation Data has correct number of files i.e {}".format(val_count))
else:
    print("Validation Data has some issue. Has following number of file : {}. Kindly Check!!".format(val_count))
    val_ok=False
if train_ok and val_ok:
    print("Dataset is Setup Correctly")

Training Data Verification
Total Number of Classes: 1000 in train directory
Total 10000 number of files in 1000 classes. i.e 10 Images/Class
Validation Data Verification
Validation Data has correct number of files i.e 50000
Dataset is Setup Correctly


### Imports

In [4]:
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.utils.data import DataLoader,Dataset

import torchvision
import torchvision.models as tvm

from torchvision import transforms
from torchvision.datasets.folder import DatasetFolder,ImageFolder

import numpy as np
from glob import glob
from PIL import Image
import pandas as pd
import os,time,gc
from pathlib import Path
from tqdm import tqdm_notebook as tqdm
import datetime,random,string


In [5]:
ngpu=torch.cuda.device_count()
device = torch.device("cuda" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

print("Using Pytorch Version : {} and Torchvision Version : {}. Using Device {}".format(torch.__version__,torchvision.__version__,device))

Using Pytorch Version : 1.4.0 and Torchvision Version : 0.5.0. Using Device cuda


### Dataset and Dataloaders Setup

In [6]:
dataset_path=r'ILSVRC/'
train_dataset_path=dataset_path+'train'
test_dataset_path=dataset_path+'valid'
print("Dataset root Folder:{}. Train Data Path: {}. Validation Data Path {}".format(dataset_path,train_dataset_path,test_dataset_path))

Dataset root Folder:ILSVRC/. Train Data Path: ILSVRC/train. Validation Data Path ILSVRC/valid


In [7]:
# Preparation of Labels 
label_dict={}
label_idx={}

with open('ILSVRC/LOC_synset_mapping.txt') as file:
    lines=file.readlines()
    for idx,line in enumerate(lines):
        label,actual =line.strip('\n').split(' ',maxsplit=1)
        label_dict[label]=actual
        label_idx[label]=idx
        

### Transforms

In [8]:
# transforms
size=224
# Imagenet Stats
vgg_mean = [103.939, 116.779, 123.68]

preprocess=transforms.Compose([transforms.Resize((size,size)),
                               transforms.ToTensor(),
                               transforms.Normalize(vgg_mean,(0.5, 0.5, 0.5))])

### Dataset and Dataloaders


In [9]:

class CustomDataset(Dataset):
    def __init__(self, subset, root_dir, transform=None):
        self.root_dir=root_dir
        self.transform=transform
       
        self.subset=subset
        if self.subset=='train':
            data_dir=os.path.join(self.root_dir,self.subset)
            self.images_fn=glob(f'{data_dir}/*/*')
            self.labels=[Path(fn).parent.name for fn in self.images_fn]
        elif subset =='valid':
            df=pd.read_csv('ILSVRC/LOC_val_solution.csv')
            df['label']=df['PredictionString'].str.split(' ',n=1,expand=True)[0]
            df=df.drop(columns=['PredictionString'])
            self.images_fn='ILSVRC/valid/'+df['ImageId'].values+'.JPEG'
            self.labels=df['label']
        else:
            raise ValueError
        print(f" Number of instances in {self.subset} subset of Dataset: {len(self.images_fn)}")       

    def __getitem__(self,idx):
        fn=self.images_fn[idx]
        label=self.labels[idx]
        image=Image.open(fn)
        if image.getbands()[0] == 'L':
            image = image.convert('RGB')
        if self.transform:
            image = self.transform(image)    
#         print(type(image))
        return image,label_idx[label]

        
    
    def __len__(self):
        return len(self.images_fn)
        
data_train=ImageFolder(root='ILSVRC/train',transform=preprocess)
class2idx=data_train.class_to_idx
data_valid=CustomDataset(subset='valid',root_dir=dataset_path,transform=preprocess)

train_num = len(data_train)
val_num = len(data_valid)



 Number of instances in valid subset of Dataset: 50000


# Proposed Approach



![Proposed approach](resources/nag.png)

- **Core idea is to model the distribution of universal adversarial perturbations for a given classifier.**
- The image shows a batch of B random vectors {z}<sub>B</sub> transforming into perturbations {delta}<sub>B</sub> by G which get added to the batch of data samples {x}<sub>B</sub>.
- The top portion shows adversarial batch (X<sub>A</sub>), bottom portion shows shuffled adversarial batch (X<sub>S</sub>) and middle portion shows the benign batch (X<sub>B</sub>). The Fooling objective Lf (eq. 2) and Diversity objective Ld (eq. 3) constitute the loss. 
### Note
- Note that the target CNN (f) is a trained classifier and its parameters are not updated during the proposed training. On the other hand, the parameters of generator (G) are randomly initialized and learned through backpropagating the loss. (Best viewed in color).

### Loss Functions/Objectives 


In [10]:

def fooling_objective(qc_):
    '''Helper function to computer compute -log(1-qc'), 
    where qc' is the adversarial probability of the class having 
    maximum probability in the corresponding clean probability
    qc' ---> qc_
    Parameters: 
    prob_vec : Probability vector for the clean batch
    adv_prob_vec : Probability vecotr of the adversarial batch
    Returns: 
    -log(1-qc') , qc'
    
    '''  
    # Get the largest probablities from predictions : Shape (bs,1)
    qc_=qc_.mean()
    return -1*torch.log(1-qc_) , qc_

def diversity_objective(prob_vec_no_shuffle, prob_vec_shuffled):
    '''Helper function to calculate the cosine distance between two probability vectors
    Parameters: 
    prob_vec : Probability vector for the clean batch
    adv_prob_vec : Probability vector for the adversarial batch
    Returns : 
    Cosine distance between the corresponding clean and adversarial batches
    '''    
    return torch.cosine_similarity(prob_vec_no_shuffle,prob_vec_shuffled).mean()

## TODO 

def intermediate_activation_objective(layer_name=None):
    ''' Extract the activations of any intermediate layer for:
    1. batch of images (of batch size=32) corrupted by the perturbations (of batch size=32) 
    2. same batch of images corrupted by same batch of perturbations but in different (random) order
    (in this case the intermdeiate layer is set to 'res4f' of ResNet 50 architecture)
    '''
    if arch =='resnet50':
        layer_name='res4f'
    
    pass


In [11]:
# Effect of ConvTranspose2d : combination of upsampling and convolution layers is equal to a strided
# convolutional layer. increase the spatial resolution of the tensor
#     def __call__(self):
# Dont Override the __call__ method. Pytorch does forward and backward hooks required, 
# Always use forward method to avoid any issues

### Generator 
- Architecture of our generator (G) unchanged for different target CNN architectures


![DCGAN](resources/DCGAN.png)

In [12]:
from torch import nn
ngf=128
nz= latent_dim=10
e_lim = 10
nc=3 # Number of Channels

# Fixed Architecture: Weights will be updated by Backprop.
class AdveraryGenerator(nn.Module):
    def __init__(self,e_lim):
        super(AdveraryGenerator, self).__init__()
        self.e_lim = e_lim
        self.main = nn.Sequential(
        nn.ConvTranspose2d( in_channels=nz,out_channels= 1024, kernel_size=4, stride=1, padding=0, bias=False),
        nn.BatchNorm2d(1024),
        nn.ReLU(True),
        # state size. (ngf*8) x 4 x 4
        nn.ConvTranspose2d(1024, 512, 4, 2, 1, bias=False),
        nn.BatchNorm2d(512),
        nn.ReLU(True),
        # state size. (ngf*4) x 8 x 8
        nn.ConvTranspose2d( 512, 256, 4, 2, 1, bias=False),
        nn.BatchNorm2d(256),
        nn.ReLU(True),
        # state size. (ngf*2) x 16 x 16
        nn.ConvTranspose2d(256, 128, 4, 2, 2, bias=False),
        nn.BatchNorm2d(128),
        nn.ReLU(True),
        # state size. (ngf) x 32 x 32
        nn.ConvTranspose2d( 128, 64, 4, 2, 2, bias=False),
        nn.BatchNorm2d(64),
        nn.ReLU(True),
        # state size. (nc) x 64 x 64
        nn.ConvTranspose2d( 64, 3, 4, 4,4, bias=False),
        nn.BatchNorm2d(3),
        nn.ReLU(True),
        nn.Tanh()
        )

    def forward(self, x):
        return self.e_lim * self.main(x) # Scaling of ε
    
    
adversarygen=AdveraryGenerator(e_lim).to(device)


#### Debugging


In [13]:
if debug:
    try:
        from torchsummary import summary
        summary(adversarygen,(nz,1,1))
    except:
        raise('Check torchsummary is installed. If not install using the command pip install torchsummary')

### Setting up Discriminator : Model : Architecture


In [14]:
from torchvision.models import googlenet, vgg16 , vgg19, resnet152, resnet50


model_dict ={
    'googlenet': googlenet,
    'vgg16': vgg16 ,
    'vgg19':vgg19, 
    'resnet152':resnet152, 
    'resnet50':resnet50 
    
}

### Choice of Hyperparameters
- The architecture of the generator consists of 5 deconv layers. The final deconv layer is followed by a tanh non-linearity and scaling by epsillon (10)


In [15]:
# Get all Pretrained Weights:
# for arch in model_dict.keys():
#     model=model_dict[arch](pretrained=True)


In [16]:
# epsillon=10
# batch_size=32
# latent_dim = 10
img_h,img_w,img_c=(224,224,3)
latent_dim=10
arch='resnet50'
archs=model_dict.keys() # ['vgg-f','vgg16','vgg19','googlenet','resnet50','resnet152'] 

def get_bs(arch):
    if torch.cuda.is_available():
#         GPU_BENCHMARK= 8192.0
#         GPU_MAX_MEM = torch.cuda.get_device_properties(device).total_memory / (1024*1024)
#         BS_DIV= GPU_BENCHMARK/GPU_MAX_MEM
#         print(f"Current GPU MAX Size : {GPU_MAX_MEM}. {BS_DIV}")

        if arch  not in ['resnet50','resnet152']:#  ['vgg16','vgg19','vgg-f','googlenet']:
            bs=int(64)
        elif arch in ['resnet50','resnet152']:
            bs=int(32)
        else:
            raise ValueError(f'Architecture type not supported. Please choose one from the following {archs}')
    else:
        bs=8 # OOM Error
    return bs

get_bs(arch)


32

In [17]:
model=model_dict[arch](pretrained=True)
model


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

### Other Utils

In [18]:
def save_checkpoint(model, to_save, filename='checkpoint.pth'):
    """Save checkpoint if a new best is achieved"""
    if to_save:
        print ("=> Saving a new best")
        torch.save(model.state_dict(), filename)  # save checkpoint
    else:
        print ("=> Validation Accuracy did not improve")
        
def save_perturbations(noise,arch,epoch,wabdb_flag=False):
    rand_str= ''.join( random.choice(string.ascii_letters) for i in range(6))
    os.makedirs(f"{arch}-{rand_str}",exist_ok=True)
    

    
    perturbations=noise.permute(0,2,3,1).cpu().detach().numpy()*255
    np.save(f'{arch}-{rand_str}/Perturbations_{arch}_{epoch}.npy', perturbations)
    for perturb_idx,perturbation in enumerate(perturbations[:,]):
        
        im = Image.fromarray(perturbation.astype(np.uint8))
        if wabdb_flag:
            wandb.log({"noise": [wandb.Image(im, caption=f"Noise_{arch}_{epoch}_{perturb_idx}")]})
        im.save(f'{arch}-{rand_str}/Perturbations_{arch}_{epoch}_{perturb_idx}.png')        

# TODO 
def visualize_perturbations():
    # MAtplotlib Subplot ?
    # Subplots(4*4) or (3*3)
    # From Memory or Disk - Epoch number ?
    pass



def get_preds(predictions,return_idx=False, k=1):
    idxs= torch.argsort(predictions,descending=True)[:,:k]
    if return_idx:
        return predictions[:,idxs], idxs
    return  predictions[:,idxs]

#### Validating Model Utils

In [19]:
# val_iterations = val_num/bs

def compute_fooling_rate(prob_adv,prob_real):
    '''Helper function to calculate mismatches in the top index vector
     for clean and adversarial batch
     Parameters:
     prob_adv : Index vector for adversarial batch
     prob_real : Index vector for clean batch
     Returns:
     Number of mismatch and its percentage
    '''
    nfool=0
    size = prob_real.shape[0]
    for i in range(size):
        if prob_real[i]!=prob_adv[i]:
            nfool = nfool+1
    return nfool, 100*float(nfool)/size      


def validate_generator_old(noise,val_dl,val_iterations=10):
    
    total_fool=0
    print("############### VALIDATION PHASE STARTED ################")
    train_log.writelines("############### VALIDATION PHASE STARTED ################")
    
    for val_idx in range(val_iterations):
        for batch_idx, data in enumerate(val_dl):
            images = data[0].to(device)
#             labels = data[1].to(device)
            
            prob_vec_clean = F.softmax(D_model(images),dim=0) # Variable q
            prob_vec_no_shuffle = D_model(images + noise)  
            nfool, _ = compute_fooling_rate(prob_vec_no_shuffle,prob_vec_clean)
            total_fool += nfool
    
    
    fool_rate = 100*float(total_fool)/(val_iterations*batch_size)       
    print(f"Fooling rate: {foolr}. Total Items Fooled :{total_fool}")
    train_log.writelines(f"Fooling rate: {foolr}. Total Items Fooled :{total_fool}")

    
    
def validate_generator(noise,D_model,val_dl):
    total_fool=0
    for batch_idx, data in tqdm(enumerate(val_dl),total = val_num//val_dl.batch_size):
        val_images = data[0].to(device)
        val_labels = data[1].to(device)

        prob_vec_clean,clean_idx = get_preds(F.softmax(D_model(val_images),dim=0),return_idx=True) # Variable q
        prob_vec_no_shuffle,adv_idx = get_preds(F.softmax(D_model(val_images + noise),dim=0),return_idx=True)  
        nfool, _ = compute_fooling_rate(adv_idx,clean_idx)
        total_fool += nfool

    fool_rate = 100*float(total_fool)/(val_num)
    return fool_rate,total_fool


    

In [20]:
## Test  Fooling Objective
adv = torch.randint(0,1000,(32,1))
real = torch.randint(0,1000,(32,1))



#### Setup Wandb

In [21]:
# Setup Wandb 

import wandb
wandb.login()
wandb.init(project="NAG_Pytorch")


W&B Run: https://app.wandb.ai/gokkulnath/NAG_Pytorch/runs/2jq8reav

# Fit and Train the Generator

In [22]:
[ random.choice(string.ascii_letters) for i in range(6)]


['O', 'b', 'E', 'A', 'F', 's']

In [23]:

def fit(nb_epochs,D_model,dls,optimizer,adversarygen=adversarygen):
    # Set the Discriminator in Eval mode; Weights are fixed.
    train_dl,val_dl = dls
    D_model=D_model.to(device)
    D_model.eval()
    timestamp=datetime.datetime.now().strftime("%d%b%Y_%H_%M")
    train_log = open(f'train_log_{arch}_{timestamp}.txt','w')
    for epoch in tqdm(range(nb_epochs),total=nb_epochs):
        running_loss=0
        rand_str= ''.join( random.choice(string.ascii_letters) for i in range(6))
        
        train_log.writelines(f"############### TRAIN PHASE STARTED : {epoch}################")
        for batch_idx, data in tqdm(enumerate(train_dl),total = train_num//train_dl.batch_size):
            # Move Data and Labels to device(GPU)
            images = data[0].to(device)
            labels = data[1].to(device)

            
            # Generate the Adversarial Noise from Uniform Distribution U[-1,1]
            latent_seed = 2 * torch.rand(bs, nz, 1, 1, device=device,requires_grad=True) -1 # (r1 - r2) * torch.rand(a, b) + r2
            noise = adversarygen(latent_seed)
            optimizer.zero_grad()

            # XB = images
            #preds_XB = f(images)
            prob_vec_clean = F.softmax(D_model(images),dim=0) # Variable q
            clean_preds ,clean_idx = get_preds(prob_vec_clean,return_idx=True,k=1)
            
            #XA = images+noise
            #preds_XA = f(images + noise)
            prob_vec_no_shuffle = D_model(images + noise)  
            qc_ =  F.softmax(prob_vec_no_shuffle,dim=0).gather(1,clean_idx) # Variable q'c

            # 1. fooling_objective: encourages G to generate perturbations that decrease confidence of benign predictions
            fool_obj, mean_qc_ = fooling_objective(qc_)
            # Perturbations  are shuffled across the batch dimesion to improve diversity
            #XS = images+ noise[torch.randperm(bs)]
            prob_vec_shuffled =   D_model(images + noise[torch.randperm(bs)])
            
            # 2.  encourages Generator to explore the space of perturbations and generate a diverse set of perturbations
            divesity_obj=diversity_objective(prob_vec_no_shuffle, prob_vec_shuffled)

            # Compute Total Loss
            total_loss = divesity_obj + fool_obj
            
            # Lets perform Backpropagation to compute Gradients and update the weights
            total_loss.backward()
            optimizer.step()
            
            # wandb Logging 
#             perturbations=noise.permute(0,2,3,1).cpu().detach().numpy()*255
#             for perturb_idx,perturbation in enumerate(perturbations[:,]):
#                 im = Image.fromarray(perturbation.astype(np.uint8))
#                 wandb.log({"noise": [wandb.Image(im, caption=f"Noise_{arch}_{epoch}_{perturb_idx}")]})
            wandb.log({"fool_obj": fool_obj.item(),
                       "divesity_obj": divesity_obj.item(),
                       "total_loss":total_loss.item(),
                      })        
            
            running_loss += total_loss.item()
            
            if batch_idx!=0  and batch_idx % 100 ==0 :
                train_log.writelines(f"############### VALIDATION PHASE STARTED : {epoch}, Step : {int(batch_idx / 100)} ################")
                fool_rate,total_fool= validate_generator(noise,D_model,val_dl)
                print(f"Fooling rate: {fool_rate}. Total Items Fooled :{total_fool}")
                train_log.writelines(f"Fooling rate: {fool_rate}. Total Items Fooled :{total_fool}")
        print(f"Diversity Loss :{divesity_obj.item()} \n Fooling Loss: {fool_obj.item()} \n")
        print(f"Total Loss after Epoch No: {epoch +1} - {running_loss/(train_num//train_dl.batch_size)}")
        train_log.writelines(f"Loss after Epoch No: {epoch +1} is {running_loss/(train_num//train_dl.batch_size)}")
        # to_save can be any expression/condition that returns a bool
        
        save_checkpoint(adversarygen, to_save= True, filename=f'GeneratorW_{arch}_{epoch}_{rand_str}.pth') 
        if epoch % 1 == 0:
#             save_perturbations(noise,arch,epoch)
            save_perturbations(noise,arch,epoch,wabdb_flag=True)
    train_log.close()

# Start Actual Trainning

In [24]:
total_epochs = 20
lr = 1e-3

# Setting up Dataloaders
import time,gc

# for arch in model_dict.keys():
arch='googlenet'
start= time.time()
print(f"Training Generator for Arch {arch}")
model= model_dict[arch](pretrained=True)
bs = get_bs(arch)
print(bs)
train_dl=DataLoader(data_train,batch_size=bs,shuffle=True,num_workers=4,pin_memory=True,drop_last=True)
val_dl=DataLoader(data_valid,batch_size=bs,shuffle=True,num_workers=4,pin_memory=True,drop_last=True)
dls = [train_dl,val_dl]
optimizer = optim.Adam(adversarygen.parameters(), lr=lr)

# del model, train_dl, val_dl,dls ,optimizer
# torch.cuda.empty_cache()
# gc.collect()
print(f"Elsasped Time {time.time()-start} Seconds")



Training Generator for Arch googlenet
64
Elsasped Time 0.18381357192993164 Seconds


In [25]:
fit(nb_epochs=total_epochs,D_model=model,dls=dls,optimizer=optimizer)

/home/ubuntu/miniconda3/envs/pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


/home/ubuntu/miniconda3/envs/pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:13: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  del sys.path[0]


/home/ubuntu/miniconda3/envs/pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:45: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



Fooling rate: 96.664. Total Items Fooled :48332

Diversity Loss :0.9685168266296387 
 Fooling Loss: 0.018528591841459274 

Total Loss after Epoch No: 1 - 1.0073822904855778
=> Saving a new best



Fooling rate: 97.31. Total Items Fooled :48655

Diversity Loss :0.8594930768013 
 Fooling Loss: 0.020314300432801247 

Total Loss after Epoch No: 2 - 0.9293945748836566
=> Saving a new best


...


Fooling rate: 96.838. Total Items Fooled :48419

Diversity Loss :0.7407758235931396 
 Fooling Loss: 0.02140353061258793 

Total Loss after Epoch No: 16 - 0.7643355555259265
=> Saving a new best



Fooling rate: 96.792. Total Items Fooled :48396

Diversity Loss :0.7163825035095215 
 Fooling Loss: 0.01921805366873741 

Total Loss after Epoch No: 17 - 0.7648925922619991
=> Saving a new best



Fooling rate: 96.502. Total Items Fooled :48251

Diversity Loss :0.7594460248947144 
 Fooling Loss: 0.020938649773597717 

Total Loss after Epoch No: 18 - 0.760070740030362
=> Saving a new best



Fooling rate: 96.504. Total Items Fooled :48252

Diversity Loss :0.7315672636032104 
 Fooling Loss: 0.015688534826040268 

Total Loss after Epoch No: 19 - 0.75711161394914
=> Saving a new best



Fooling rate: 96.138. Total Items Fooled :48069

Diversity Loss :0.7479978799819946 
 Fooling Loss: 0.01563749462366104 

Total Loss after Epoch No: 20 - 0.7516315950032992
=> Saving a new best



# Misc:


In [26]:
#  with tqdm(total = len(traindata)) as epoch_pbar:
#                 epoch_pbar.set_description(f'Epoch {epoch}')
# https://discuss.pytorch.org/t/training-a-model-via-a-train-method/58567

##### Setup Caffenet and VGG-F  (TODO)


#### Loading VGG-F

#### To do : Need to Write a Custom Model from scratch for VGG -F  and load weights from caffe model
- Refer Here : https://github.com/val-iisc/nag/blob/83564eb4a8b5177660e2f6566dd63faa16f76773/nets/vgg_f.py
- https://github.com/val-iisc/nag/blob/83564eb4a8b5177660e2f6566dd63faa16f76773/misc/convert_weights.py
- Here VGGF refers to VGG-Face model  http://www.vlfeat.org/matconvnet/pretrained/. 
- How we can use that to classify Imagenet ?
- load caffe prototxt and weights directly in pytorch --> https://github.com/marvis/pytorch-caffe
- Convert Caffe models to Pytorch : https://github.com/vadimkantorov/caffemodel2pytorch
#### Check this link for Conversion Tutorial : [Link](https://colab.research.google.com/drive/1i2dq6qctPvrLREhKOZNNBsNfKuaS0HYQ)

### Downloading Trained Weights from Google Drive  (TODO)


In [27]:
# #TODO
# !pip install gdown
# import gdown
# import zipfile

# url = 'https://drive.google.com/uc?id=0B9P1L--7Wd2vU3VUVlFnbTgtS2c' # Need to update actual URL
# output = 'weights.zip'
# gdown.download(url, output, quiet=True)

# zipdata = zipfile.ZipFile('weights.zip')
# zipinfos = zipdata.infolist()
# for zipinfo in zipinfos:
#     zipdata.extract(zipinfo)

### Evaluating NAG performance across Models: For Tabular Column Generation


# Question to be Answered 
- In the Repo code train_generator.py 
L241 : feature_loss = -10*tf.reduce_mean(tf.squared_difference(f1_res4f,f2_res4f))#feature_distance(f1_res4f,f2_res4f)
# Why is it computed and multiplied by 10 ?


### Steps to evaluate the perturbations generated by Generator Network (TODO)
arch='Fixed'
for modelarch, model in model_dict.items():
    num_iteration = 10 # Blackbox Settings
    if modelarch == arch:
        num_iteration =100 # Whitebox Settings 
    for i range(num_iteration)
        1. Load the Weights of the Generator
        2. Generate a Perturbation using a random vector of dimension latent_dim,1
        3. Add the noise to a sample image 


## References:
- GAN Architecture : Pytorch Tutorial
- [Transpose Convolution Docs](https://pytorch.org/docs/stable/nn.html?highlight=convtranspose2d#torch.nn.ConvTranspose2d)

In [28]:

# def fit(nb_epochs,D_model,dls,optimizer,adversarygen=adversarygen):
#     # Set the Discriminator in Eval mode; Weights are fixed.
#     train_dl,val_dl = dls
#     D_model=D_model.to(device)
#     D_model.eval()
#     train_log = open(f'train_log_{arch}.txt','w')
#     for epoch in tqdm(range(nb_epochs),total=nb_epochs):
#         running_loss=0
#         print(f"############### TRAIN PHASE STARTED : {epoch} ################")
#         train_log.writelines(f"############### TRAIN PHASE STARTED : {epoch}################")
#         tic=time.time()
#         for batch_idx, data in tqdm(enumerate(train_dl),total = train_num//train_dl.batch_size):
#             # Move Data and Labels to device(GPU)
#             images = data[0].to(device)
#             labels = data[1].to(device)
            
            
#             # Generate the Adversarial Noise from Uniform Distribution U[-1,1]
#             latent_seed = 2 * torch.rand(bs, nz, 1, 1, device=device,requires_grad=True) -1 # (r1 - r2) * torch.rand(a, b) + r2
#             noise = adversarygen(latent_seed)
#             optimizer.zero_grad()

#             # XB = images
#             #preds_XB = f(images)
#             prob_vec_clean = F.softmax(D_model(images),dim=0) # Variable q
#             _ ,clean_idx = get_preds(prob_vec_clean,return_idx=True,k=1)
            
#             #XA = images+noise
#             #preds_XA = f(images + noise)
#             prob_vec_no_shuffle = D_model(images + noise)  
#             qc_ =  F.softmax(prob_vec_no_shuffle,dim=0)[:,clean_idx] # Variable q'c

#             # 1. fooling_objective: encourages G to generate perturbations that decrease confidence of benign predictions

#             fool_obj, mean_qc_ = fooling_objective(qc_)

#             # Perturbations  are shuffled across the batch dimesion to improve diversity
#             #XS = images+ noise[torch.randperm(bs)]
#             prob_vec_shuffled =   D_model(images + noise[torch.randperm(bs)])
            
#             # 2.  encourages Generator to explore the space of perturbations and generate a diverse set of perturbations
#             divesity_obj=diversity_objective(prob_vec_no_shuffle, prob_vec_shuffled)

#             # Compute Total Loss
#             total_loss = divesity_obj+fool_obj

#             # Lets perform Backpropagation to compute Gradients and update the weights
#             total_loss.backward()
#             optimizer.step()
            
#             running_loss += total_loss.item()
            
#             if batch_idx!=0 and batch_idx % 100 ==0 :
#                 print(f"############### VALIDATION PHASE STARTED : {epoch}, Step :{int(batch_idx / 100)} ################")
#                 train_log.writelines(f"############### VALIDATION PHASE STARTED : {epoch}, Step :{int(batch_idx / 100)} ################")
#                 ticval=time.time()
#                 fool_rate,total_fool= validate_generator(noise,D_model,val_dl)
#                 print(f"Fooling rate: {fool_rate}. Total Items Fooled :{total_fool}")
#                 train_log.writelines(f"Fooling rate: {fool_rate}. Total Items Fooled :{total_fool}")
#                 print(f"Time Elasped for Validating: {time.time()-ticval} Seconds")

#         print(f"Loss after Epoch No: {epoch +1} is {running_loss}")
#         train_log.writelines(f"Loss after Epoch No: {epoch +1} is {running_loss}")
#         # to_save can be any expression/condition that returns a bool
#         save_checkpoint(adversarygen, to_save= True, filename=f'GeneratorW_{arch}_{epoch}.pth') 
#         if epoch % 1 == 0:
#             save_perturbations(noise,arch,epoch)
#         print(f"Time Elasped for Trainning one Epoch : {time.time()-tic} Seconds")
#     train_log.close()